# 第2回：転移学習と不均衡データへの挑戦

## 1. 目的
- 皮膚病変画像（DermaMNIST）を用い、実務上の課題である「データの偏り」への対策を学ぶ。
- 学習済みモデルを用いた「転移学習」により、高精度なモデルを構築する。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
import numpy as np
from sklearn.utils import class_weight

# リポジトリをクローン
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

sys.path.insert(0, '/tmp/MedMNIST-Exercise')

# モジュールをインポート
sys.modules.pop('exercise_logic', None)
import exercise_logic

exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

# 1. データのロード（DermaMNIST）
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('dermamnist')

# 2. クラス不均衡の計算
y_train_flat = y_train.flatten()
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_flat), y=y_train_flat)
class_weights_dict = dict(enumerate(weights))

print("算出されたクラス重み:", class_weights_dict)

In [ ]:
# 3. 転移学習モデル（MobileNetV2）を構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='transfer'
)

# 4. クラス重みを適用して学習
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    class_weight=class_weights_dict, 
    batch_size=32
)

In [ ]:
# 5. 評価
exercise_logic.plot_history(history)
exercise_logic.show_evaluation_reports(model, x_test, y_test, info['label'])